# Assignment 2: Binary and Categorical Classification

For this assignment, we will use a similar two-layer network as Assignment 1 to implement binary and categorical classification using only `NumPy` package, without relying on `PyTorch` or any deep learning libraries.

We will implement Binary Cross Entropy Loss and Categorical Cross Entropy Loss and their gradient manually with efficient matrix computation, and evaluate accuracy for two classification tasks.  


## Theoretical Questions

We will build and train a two-layer network similar to Assignment 1.
Besides, we will apply **Logistic Function** ($\sigma$) on the logits for binary classification and **Stable Softmax Function** ($\tilde{\mathcal S}$) on the logits for categorical classification.

Questions in this section concern:

1. Comparison between normal softmax $\mathcal S$ and stable softmax $\tilde{\mathcal S}$
2. Compute derivative of $\tilde{\mathcal S}$ that should be applied in the coding section
3. Weight/Parameter Initialization
4. Discussion on Evaluation Metrics of classification tasks

### Task 1.1 Proof of Stable Softmax

The softmax function is defined as:

$$
\mathcal S(z_o) = \frac{e^{z_o}}{\sum\limits_{o'} e^{z_{o'}}}
$$

However, when $ z_o $ has large values, the exponential $ e^{z_o} $ may grow too large, leading to numerical instability (e.g., overflow issues).
To improve stability, we introduce the **stable softmax**:

$$
\tilde{\mathcal S}(z_o) = \frac{e^{z_o - z_{\max}}}{\sum\limits_{o'} e^{z_{o'} - z_{\max}}}
$$

where $z_{\max} = \max\limits_o z_o$.

Prove that stable softmax is mathematically equivalent to the original softmax function.

Discuss how stable softmax improves numerical stability, for instance, what will happen when $z_o$ is too large, or too small?

Answer:

$$\begin{aligned}
\tilde{\mathcal S}(z_o) &= \frac{e^{z_o - z_{\max}}}{\sum\limits_{o'} e^{z_{o'} - z_{\max}}}\\[6ex] 
&= \frac{e^{z_o}e^{-z_{\max}}}{e^{-z_{\max}}\sum\limits_{o'} e^{z_{o'}}} \\[6ex] 
&= \frac{e^{z_o}} {\sum\limits_{o'} e^{z_{o'}}} = \mathcal S(z_o)
\end{aligned}$$

**Prevents Overflow**: If $ z_o $ is large, $ e^{z_o} $ can exceed the floating-point limit, leading to numerical overflow. The stable form ensures all exponentiated terms remain within a reasonable range.

**Prevents Underflow**: If $ z_o $ is very small, $ e^{z_o} $ may be rounded to zero, causing division by a near-zero denominator. Stability ensures a well-defined denominator.


### Task 1.2 Derivative of Stable Softmax


Recall that in the lecture slides, when we compute gradient with respect to $w^{(1)}$ and $w^{(2)}$, as part of the chain rule, we replace $\frac{\partial\mathcal J^{[n]}}{\partial\mathcal y_o^{[n]}}\frac{\partial\mathcal y_o^{[n]}}{\partial\mathcal z_o^{[n]}}$ by $\frac{\partial\mathcal J^{[n]}}{\partial\mathcal z_o^{[n]}}$ when we choose a good activation function.

When normal softmax is applied, we have $\frac{\partial\mathcal J^{[n]}}{\partial\mathcal z_o^{[n]}} = y_o^{[n]} - t_o^{[n]}$. Prove that $\frac{\partial\mathcal J^{[n]}}{\partial\mathcal z_o^{[n]}}$ will return the same format when the activation function is stable softmax.



Answer:

$$\begin{aligned}
\mathcal J^{\text{CCE}}
&= -\frac{1}{N} \sum_{n=1}^{N} \log y_{\tau^{[n]}}^{[n]}
&= -\frac{1}{N} \sum_{n=1}^{N} \log \frac{e^{z_{\tau^{[n]}}^{[n]} - z_{\max}^{[n]}}}{\sum_{o=1}^{O} e^{z_o^{[n]} - z_{\max}^{[n]}}}  
\end{aligned}$$

$$\begin{aligned}
\frac{\partial\mathcal J^{[n]}}{\partial\mathcal z_o^{[n]}}
&= -\frac{\partial \Bigl[ \sum_{o'=1}^{O} t_{o'}^{[n]}({z_{o'}^{[n]} - z_{\max}^{[n]}}) - \log \sum_{o'=1}^{O} e^{{z_{o'}^{[n]} - z_{\max}^{[n]}}}\Bigr]}{\partial z_o^{[n]}}\\[6ex]
&= -\frac{\partial \sum_{o'=1}^{O} t_{o'}^{[n]}({z_{o'}^{[n]} - z_{\max}^{[n]}})}{\partial z_o^{[n]}} + \frac{\partial \log \sum_{o'=1}^{O} e^{{z_{o'}^{[n]} - z_{\max}^{[n]}}}}{\partial z_o^{[n]}}\\[6ex]
&=-\frac{\partial t_o^{[n]}({z_o^{[n]} - z_{\max}^{[n]}})}{\partial z_o^{[n]}} + \frac{1}{\sum_{o'=1}^{O} e^{{z_{o'}^{[n]} - z_{\max}^{[n]}}}}\frac{\partial \sum_{o'=1}^{O} e^{{z_{o'}^{[n]}-z_{\max}^{[n]}}}}{\partial z_o^{[n]}}\\[6ex]
&= -t_o^{[n]} + \frac{e^{z_o^{[n]}-z_{\max}^{[n]}}}{\sum_{o'=1}^{O} e^{{z_{o'}^{[n]} - z_{\max}^{[n]}}}}
= y_o^{[n]} -t_o^{[n]} 
\end{aligned}$$

### Task 1.3 Weight Initialization

Why do we need to randomly initialize the weights before training starts?

1. Proof that a network with constantly initialized weights perform like a network with one hidden unit.
2. Show that gradient decent is not able to change this behavior and will keep all neurons in the hidden layer identical throughout training.

Answer:

Assume: all weights are initialized identically:

$$\forall d,k \colon w_{k,d}^{(1)} = w_k^{(2)} =  w$$

When we compute the activation $a_k$:

$$a_k = \sum\limits_{d=0}^D w_{k,d}^{(1)} x_d = w \sum\limits_{d=0}^D x_d$$

Thus, the right-hand side is independent of $k$.
Since all activations are identical, the network acts like a single hidden unit.

Also all $h_k = g(a_k) = h$ are identical.

---


Does the weight update improve this behavior?
Let's have a look at the derivatives:
$$\frac{\partial \mathcal J}{\partial w_{k,d}^{(1)}}=\frac{2}{N}\sum_{n=1}^{N}(y^{[n]}-t^{[n]})w_k^{(2)}h_k^{[n]}(1-h_k^{[n]})x_d^{[n]}=\frac{2}{N}\sum_{n=1}^{N}(y^{[n]}-t^{[n]})wh^{[n]}(1-h^{[n]})x_d^{[n]}$$


Again the final formula is independent of $k$.
Thus, the weight updates for all hidden neurons are identical.
Still all weights are identical and it acts like a single hidden unit.


### Task 1.4 Evaluation Metrics

If a binary classifier achieves an accuracy greater than 90%, see example plot in the OLAT, does this always indicate good performance? Might high accuracy in this binary classification be misleading? 

What alternative evaluation approaches could better capture model performance in such scenarios?



Answer:

In highly unbalanced data, looking at only accuracy is not a good way to capture the model performance. Also, when the context of the problem (churn) is considered, companies does not want their customer to churn. So, they are more interested in people who stop using their products. To evaluate that, it would be better to include another evaluation metric like TPR.